In his PhD thesis on GA optimisation of trading Lidquist uses the following strategy:

![Lindquist-gene-encoding](images/Lindquist-gene-encoding01.png)
![Lindquist-gene-encoding](images/Lindquist-gene-encoding02.png)
![Lindquist-gene-encoding](images/Lindquist-gene-encoding03.png)
![Lindquist-gene-encoding](images/Lindquist-gene-encoding04.png)

